In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
%matplotlib inline

**Lets Read The Data**

In [ ]:
train=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-5/train.csv")
test=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-5/test.csv")

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

**Saving the Id For Furture Use**

In [ ]:
ID=train["Id"]
FID=test["ForecastId"]

**Dropping Attributes Which Has Many Null Values**

In [ ]:
train=train.drop(columns=['County','Province_State','Id'])
test=test.drop(columns=['County','Province_State','ForecastId'])

# Data Visualisations

In [ ]:
sns.pairplot(train)

**The above graph shows only the numerical features**

**Target Vs Target Value**

In [ ]:
sns.barplot(x="Target",y="TargetValue",data=train)

**Population vs target Value**

In [ ]:
sns.barplot(x='Target',y='Population',data=train)

**A Pie Chart To Show Country_Region vs TargetValue**

In [ ]:
fig=plt.figure(figsize=(45,30))
fig=px.pie(train, values='TargetValue', names='Country_Region')
fig.update_traces(textposition='inside')
#fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

# Exploratory Data Analysis

In [ ]:
#We Group Or Add Every Rows With Same Country Region
df_grouped=train.groupby(['Country_Region']).sum()
df_grouped.TargetValue

**Top 5 countries with the higest target value!**

In [ ]:
top5=df_grouped.nlargest(5,'TargetValue')
top5

**Plotting Top 5 Target Values vs Its Population**

In [ ]:
sns.catplot(y="Population", x="TargetValue",kind="bar", data=top5)
plt.title('Top 5 Target Values',size=20)
plt.show()

**Top 5 Countries With Highest Population**

In [ ]:
top5pop=df_grouped.nlargest(5,'Population')
top5pop

**Visualizing Treemaps(nested rectangles)in terms of population,target value of every country. Each group is represented by a rectangle, which area is proportional to its value. Using color schemes, it is possible to represent several dimensions: groups, subgroups**

In [ ]:
fig = px.treemap(train, path=['Country_Region'], values='TargetValue',
                  color='Population', hover_data=['Country_Region'],
                  color_continuous_scale='matter')
fig.show()

**In the above plot the color represents population and the area of the sqaure represents target value**

**Coverting Date to Int format in both test and train dataset!**

In [ ]:
#If ‘coerce’, then invalid parsing will be set as NaN.
#strftime=Return an Index of formatted strings specified by date

dateee= pd.to_datetime(train['Date'], errors='coerce')
train['Date']= dateee.dt.strftime("%Y%m%d").astype(int)
dateee= pd.to_datetime(test['Date'], errors='coerce')
test['Date']= dateee.dt.strftime("%Y%m%d").astype(int)

**Creating heatmaps for top 2000 entries in terms of Target Value**

In [ ]:
top2000=train.nlargest(2000,'TargetValue')
top2000

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

h=pd.pivot_table(top2000,values='TargetValue',index=['Country_Region'],columns='Date')

sns.heatmap(h,cmap="RdYlGn",linewidths=0.05)

**Green Means More And Red Means Less**

**So a lot can be depicted from this heatmap we see how it all started in china and slowly infected iran,italy,spain and finally the US in the worse manner possible**

**Lets now plot top 2000 populated states heatmap**

In [ ]:
top2000pop=train.nlargest(2000,'Population')
top2000pop

In [ ]:
fig ,ax=plt.subplots(figsize=(20,10))
h=pd.pivot_table(top2000pop,values='TargetValue',index=['Country_Region'],columns="Date")
sns.heatmap(h,cmap="RdYlGn",linewidths=0.005)

**From above heatmap we can see US is in a bad state and Brazil And Indias Count Is Slowly Rising**

# Data Processing

Lets Inititalize Cat Features of Both train and test

In [ ]:
train.info()

**We see that the obj values are in row 0 and row 4 so we will label encode those features**

In [ ]:
from sklearn.preprocessing import LabelEncoder
enco = LabelEncoder()

temp=train.iloc[:,0].values
train.iloc[:,0]=enco.fit_transform(temp)

temp=train.iloc[:,4].values
train.iloc[:,4]=enco.fit_transform(temp)

In [ ]:
train.info()

**We Do The Same For Test Dataset**

In [ ]:
test.info()

In [ ]:
temp=test.iloc[:,0].values
test.iloc[:,0]=enco.fit_transform(temp)

temp=test.iloc[:,4].values
test.iloc[:,4]=enco.fit_transform(temp)

In [ ]:
test.info()

# Modelling

In [ ]:
target=train["TargetValue"]
df_train=train.drop(["TargetValue"],axis=1)

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df_train,target,test_size=0.3,random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

**Lets Make Prediction**

In [ ]:
pipe = Pipeline([("scaler2",StandardScaler()),("RandomForestRegressor:",RandomForestRegressor())])

pipe.fit(x_train,y_train)

prediction=pipe.predict(x_test)

In [ ]:
acc=pipe.score(x_test,y_test)
acc

In [ ]:
predict=pipe.predict(test)

In [ ]:
output=pd.DataFrame({'id':FID,'TargetValue':predict})
output

# Now we edit the output according to the problem statement

*Quantiles are points in a distribution that relate to the rank order of values in that distribution.
For a sample, you can find any quantile by sorting the sample. The middle value of the sorted sample (middle quantile, 50th percentile) is known as the median. The limits are the minimum and maximum values. Any other locations between these points can be described in terms of centiles/percentiles.*

**Lets Implements Quantile Via Numpy**

In [ ]:
a=output.groupby(['id'])['TargetValue'].quantile(q=0.05).reset_index()
b=output.groupby(['id'])['TargetValue'].quantile(q=0.5).reset_index()
c=output.groupby(['id'])['TargetValue'].quantile(q=0.95).reset_index()

**Bringing all the calculated values into the same dataframe i.e values of q0.05 , q0.5 , 0.95 !!**

In [ ]:
a.columns=['Id','q0.05']
b.columns=['Id','q0.5']
c.columns=['Id','q0.95']

total=pd.concat([a,b['q0.5'],c['q0.95']],axis=1)


total

*To make analysis of data in table easier, we can reshape the data into a more computer-friendly form using Pandas in Python. 
Pandas.melt() is one of the function to do so..
Pandas.melt() unpivots a DataFrame from wide format to long format.*

*melt() function is useful to massage a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are unpivoted to the row axis, leaving just two non-identifier columns, variable and value*

In [ ]:
sub=pd.melt(total, id_vars=['Id'], value_vars=['q0.05','q0.5','q0.95'])

print(sub)

In [ ]:
#Removing the character "q"
sub['variable']=sub['variable'].str.replace("q","", regex=False)
#Formating ForecastId with in this format "Id_variable"
sub['ForecastId_Quantile']=sub['Id'].astype(str)+'_'+sub['variable']

sub['TargetValue']=sub['value']

In [ ]:
sub=sub[['ForecastId_Quantile','TargetValue']]
sub.reset_index(drop=True,inplace=True)


sub.to_csv("submission.csv",index=False)

In [ ]:
sub